In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras


from keras.layers import Dense
from keras.models import Sequential,Model


from tensorflow.keras.optimizers import Adam

In [2]:
class GaussianNoise(tf.keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super(GaussianNoise, self).__init__(**kwargs)
        self.stddev = stddev

    def call(self, inputs, training=None):
        if 1 or training:
            noise = tf.random.normal(tf.shape(inputs), stddev=self.stddev)
            return inputs + noise
        return inputs

    def get_config(self):
        config = super(GaussianNoise, self).get_config()
        config.update({'stddev': self.stddev})
        return config
    
    
class L2Normalization(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        
        super(L2Normalization, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.nn.l2_normalize(inputs, axis=-1)

    def get_config(self):
        return super(L2Normalization, self).get_config()
    

def generate_nakagami_samples(m, omega, shape):

    return tf.random.gamma(shape, m, 1/omega) ** 0.5
    
class NakagamiNoiseLayer(tf.keras.layers.Layer):
    def __init__(self, distribution_params, **kwargs):
        super(NakagamiNoiseLayer, self).__init__(**kwargs)
        self.distribution_params = distribution_params

    def call(self, inputs, training=None):
        if  1 or training:
            # noise = tf.random.normal(tf.shape(inputs), **self.distribution_params)
            if tf.shape(inputs)[0] == None:
                noise = generate_nakagami_samples(m = self.distribution_params["m"], 
                                              omega = self.distribution_params["omega"], 
                                              shape = tf.shape(inputs))
            else:
                noise = generate_nakagami_samples(m = self.distribution_params["m"], 
                                              omega = self.distribution_params["omega"], 
                                              shape = tf.shape(inputs)[1:])
            return inputs * noise
        
        else:
            return inputs
        
        





In [3]:
# generating the data set
k = 7 #3
M = 2**k


n_train = 1000 * M
n_val = 100 * M

x_train = np.array(np.random.rand(n_train,k)<0.5).astype(np.float32)
y_train = x_train


x_val = np.array(np.random.rand(n_val,k)<0.5).astype(np.float32)
y_val = x_val



In [4]:
accuracies = []
models = []
attempts = 0
while True:
        attempts += 1
        print(f"-------  start of attempt : {attempts} ----------")
        AE = Sequential([
            

                                Dense(2*k, activation='tanh',input_shape=(k,)),
                                Dense(2*k, activation='tanh'),

                                Dense(2, activation='linear'),
                                L2Normalization(name="normalization_layer"),


                                NakagamiNoiseLayer({"omega":1,"m":5}),
                                GaussianNoise(stddev=0.51,name="channel"),

                                L2Normalization(name="normalization_layer_at_rx"),

                                Dense(2*k, activation='tanh'),
                                Dense(k, activation='sigmoid')



                        ])

        AE.build()
        #     AE.summary()
        
        AE.compile(optimizer=Adam(learning_rate=1e-2),loss="binary_crossentropy")
        AE.fit(x_train,y_train,epochs=10,)
        
        preds = AE.predict(x_val)>0.5
        accuracy = np.mean( preds == y_val  )
        print(f"accuracy = {accuracy}")
        
        accuracies.append(accuracy)
        models.append(AE)
        
        if accuracy > 0.97 or attempts >= 5:
            break
        
        
        
print("Accuracies: ", accuracies)

-------  start of attempt : 1 ----------
Epoch 1/10
4000/4000 [==============================] - 7s 2ms/step - loss: 0.4751
Epoch 2/10
4000/4000 [==============================] - 6s 1ms/step - loss: 0.4653
Epoch 3/10
4000/4000 [==============================] - 6s 2ms/step - loss: 0.4658
Epoch 4/10
4000/4000 [==============================] - 6s 2ms/step - loss: 0.4645
Epoch 5/10
4000/4000 [==============================] - 6s 1ms/step - loss: 0.4632
Epoch 6/10
4000/4000 [==============================] - 6s 2ms/step - loss: 0.4645
Epoch 7/10
4000/4000 [==============================] - 6s 2ms/step - loss: 0.4640
Epoch 8/10
4000/4000 [==============================] - 6s 2ms/step - loss: 0.4657
Epoch 9/10
4000/4000 [==============================] - 6s 1ms/step - loss: 0.4638
Epoch 10/10
400/400 [==============================] - 0s 827us/step
accuracy = 0.6879799107142858
-------  start of attempt : 2 ----------
Epoch 1/10
4000/4000 [==============================] - 6s 1ms/step - lo

In [ ]:
# select the best out of all the AE models saved in AE
bestIndex = np.argmax(accuracies)
print("best accuracy: ", max(accuracies))
AE_best = models[bestIndex]

In [ ]:
# bc = before_channel.predict(np.array([
    
#     [0,0],
#     [0,1],
#     [1,0],
#     [1,1]
  
    
# ]))



# bc = before_channel.predict(np.array([

#             [0,0,0,0],
#             [0,0,0,1],
#             [0,0,1,0],
#             [0,0,1,1],
#             [0,1,0,0],
#             [0,1,0,1],
#             [0,1,1,0],
#             [0,1,1,1],
#             [1,0,0,0],
#             [1,0,0,1],
#             [1,0,1,0],
#             [1,0,1,1],
#             [1,1,0,0],
#             [1,1,0,1],
#             [1,1,1,0],
#             [1,1,1,1]
    
# ]))




In [ ]:


before_channel = Model(inputs=AE_best.input,
                                 outputs=AE_best.get_layer('normalization_layer').output)

after_channel  = Model(inputs=AE_best.input,
                                 outputs=AE_best.get_layer('channel').output)

after_channel_normalized = Model(inputs=AE_best.input,
                                 outputs=AE_best.get_layer('normalization_layer_at_rx').output)
    
    
bc = before_channel.predict(np.array([
    [0,0,0],
    [0,0,1],
    [0,1,0],
    [0,1,1],
    [1,0,0],
    [1,0,1],
    [1,1,0],
    [1,1,1],
    
  
  
    
]))
    

ac = after_channel.predict(x_val[:1000,:])
acn = after_channel_normalized.predict(x_val[:1000,:])


fig,ax = plt.subplots(1,3,figsize=(15,4))
ax[0].scatter(bc[:,0],bc[:,1],c="red")
ax[0].set_title("before channel")


ax[1].scatter(ac[:,0],ac[:,1],c="blue",s=3)
ax[1].set_title("after channel")


ax[2].scatter(ac[:,0],ac[:,1],c="blue",s=3)
ax[2].scatter(bc[:,0],bc[:,1],c="red") # seconded to avoid occlusion
ax[2].set_title("both before and after channel normalized")


plt.show()

In [ ]:



# # Example usage:
# m = 1.5  # Shape parameter
# omega = 2.0  # Scale parameter
# num_samples = 1000
# nakagami_samples = generate_nakagami_samples((None,num_samples,),m, omega)


# # Compute the histogram
# hist, bins = np.histogram(nakagami_samples, bins=20)  # Adjust the number of bins as needed

# # Plot the histogram
# plt.figure(figsize=(8, 6))
# plt.hist(nakagami_samples, bins=20, color='blue', alpha=0.7)  # Plot the histogram bars
# plt.xlabel('Value')
# plt.ylabel('Frequency')
# plt.title('Histogram of Nakagami Samples')
# plt.grid(True)
# plt.show()

In [ ]:
bc

In [ ]:
class RicianFadingLayer(tf.keras.layers.Layer):
    def __init__(self, K, stddev, **kwargs):
        super(RicianFadingLayer, self).__init__(**kwargs)
        self.K = K  # Rician factor (K-factor)
        self.stddev = stddev  # Standard deviation of Rayleigh fading

    def call(self, inputs):
        # Generate Rayleigh fading components
#         rayleigh = tf.random.normal(tf.shape(inputs), mean=0.0, stddev=self.stddev)
        rayleigh = tf.random.rayleigh(shape=tf.shape(inputs),scale=self.stddev)
    
        # Generate line-of-sight (LOS) component
        LOS = tf.sqrt(self.K / (self.K + 1)) * tf.random.normal(tf.shape(inputs), mean=0.0, stddev=self.stddev)
        
        # Combine LOS and Rayleigh components
        rician = LOS + rayleigh
        # Apply Rician fading to input signals
        return inputs * rician

# # Example usage:
# # Create a Rician fading layer with K-factor 3 and standard deviation 0.1
# rician_layer = RicianFadingLayer(K=3, stddev=0.1)

# # Generate some input signals
# inputs = tf.random.normal((32, 64))  # Example input shape: (batch_size, input_dim)

# # Apply Rician fading to the input signals
# outputs = rician_layer(inputs)

In [ ]:
preds = AE_best.predict(x_val) > 0.5
accuracy = np.mean(  preds == y_val  )


print(f"accuracy = {accuracy}")
        

In [6]:
y_val.shape


(12800, 7)